<a href="https://colab.research.google.com/github/Abu5005one/Exam-prep-bot/blob/main/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate bitsandbytes transformers unsloth datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.5/166.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments,DataCollatorForLanguageModeling
from trl import SFTTrainer
import torch
from datasets import load_dataset
import os

In [10]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 512,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [11]:

dataset = load_dataset("json", data_files="interview_prep.jsonl", split="train")

In [12]:
tokenizer.chat_template = (
    "{% for message in messages %}"
    "{% if message['role'] == 'user' %}"
    "{{ bos_token + 'User: ' + message['content'] + eos_token }}"
    "{% elif message['role'] == 'assistant' %}"
    "{{ bos_token + 'Assistant: ' + message['content'] + eos_token }}"
    "{% endif %}"
    "{% endfor %}"
)

In [13]:
# 4. Apply formatting and tokenize
dataset = dataset.map(lambda example: {
    "text": tokenizer.apply_chat_template(example["messages"], tokenize=False)
})
dataset = dataset.map(lambda x: tokenizer(x["text"]), batched=False)

In [14]:
# 5. Training arguments
args = TrainingArguments(
    output_dir = "llama_interview_model",
    per_device_train_batch_size = 1,
    num_train_epochs = 1,
    learning_rate = 2e-4,
    logging_steps = 1,
    save_strategy = "no",
    optim = "adamw_8bit",
    fp16 = True,
)

In [15]:
# 6. Apply PEFT (LoRA)
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none"
)

os.environ["WANDB_MODE"] = "disabled"

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.7.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [16]:

#  Fine-tune
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="input_ids",
    max_seq_length=512,
    args=args,
    packing=False,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,3.025300
2,3.734400
3,2.466000
4,3.187700
5,2.755800
6,2.596400
7,1.815100
8,2.242000
9,1.565500
10,1.954500


TrainOutput(global_step=10, training_loss=2.534265625476837, metrics={'train_runtime': 14.9895, 'train_samples_per_second': 0.667, 'train_steps_per_second': 0.667, 'total_flos': 18927544221696.0, 'train_loss': 2.534265625476837})

In [17]:

# 8. Save
model.save_pretrained("llama_interview_model")
tokenizer.save_pretrained("llama_interview_model")

('llama_interview_model/tokenizer_config.json',
 'llama_interview_model/special_tokens_map.json',
 'llama_interview_model/chat_template.jinja',
 'llama_interview_model/tokenizer.json')

In [18]:
prompt = "User: Why do you want to work in the field of AI?\nAssistant:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True, top_k=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

User: Why do you want to work in the field of AI?
Assistant: I am passionate about the potential of AI to transform industries and improve people’s lives.
